This notebook illustrates local search with insertion implementation step by step. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
import utils

In [3]:
np.set_printoptions(legacy='1.25')              # set print options

<Token var=<ContextVar name='format_options' default={'edgeitems': 3, 'threshold': 1000, 'floatmode': 'maxprec', 'precision': 8, 'suppress': False, 'linewidth': 75, 'nanstr': 'nan', 'infstr': 'inf', 'sign': '-', 'formatter': None, 'legacy': 9223372036854775807, 'override_repr': None} at 0x71bb80086ed0> at 0x71bb78222b00>

In [4]:
df = pd.read_csv('../data/connectome_graph.csv')
sol_df = pd.read_csv('../data/benchmark.csv')   # first read in as a pd.DataFrame
sol = sol_df.set_index('Node ID')['Order']      # convert to pd.Series

In [5]:
sol = sol.sort_values()                         # solution needs to be sorted by order 

In [6]:
sol[:5]

Node ID
720575940626087322    0
720575940611746258    1
720575940610412386    2
720575940624724919    3
720575940626365768    4
Name: Order, dtype: int64

In [7]:
score = utils.evaluate(df, sol)
score

29023882

In [8]:
# Convert DataFrame columns to NumPy arrays
source_node_ids = df['Source Node ID'].to_numpy()
target_node_ids = df['Target Node ID'].to_numpy()
edge_weights = df['Edge Weight'].to_numpy()
sol_array = sol.index.to_numpy()

In [9]:
d_weights = dict(zip(zip(df['Source Node ID'], df['Target Node ID']), df['Edge Weight']))  # for faster lookup

In [10]:
total = sum(d_weights.values())
total

41912141

In [11]:
node_id = 720575940619500248

In [12]:
nodes_subset = utils.extract_subgraph(source_node_ids, target_node_ids, node_id) # identify node ids of the nodes in the subgraph

In [13]:
subset_indices = np.nonzero(np.isin(sol_array, nodes_subset))[0]                 # get the order of the nodes in the subgraph

In [14]:
sol_subset = sol_array[subset_indices]               # node ids of the subgraph in the solution order
sol_subset_keep = sol_subset.copy()

i = np.where(sol_subset == node_id)[0][0]            # order of the current node
d_left = utils.swap_left(sol_subset, i, d_weights)   # difference by swapping with nodes on the left
d_right = utils.swap_right(sol_subset, i, d_weights) # difference by swapping with nodes on the right
d_diff = {**d_left, **d_right, node_id: 0} 

In [16]:
d_diff

{720575940627873822: 10,
 720575940630985723: 18,
 720575940644126664: 38,
 720575940613978065: 112,
 720575940631590907: 110,
 720575940621838320: -57,
 720575940613865443: -64,
 720575940617837798: -64,
 720575940624682218: -94,
 720575940626977636: -92,
 720575940639047908: -85,
 720575940616012138: -80,
 720575940619500248: 0}

In [17]:
insert_partner = max(d_diff, key=d_diff.get)                                    # node ID for insertion partner
diff = d_diff[insert_partner]                                                   # difference in score of the insertion
max_loc = subset_indices[np.where(sol_subset_keep == insert_partner)[0][0]]     # index of the insertion location

In [18]:
diff

112

In [19]:
max_loc

13067

In [21]:
# Insertion
i = subset_indices[np.where(sol_subset_keep == node_id)[0][0]]                  # current node's order
j = max_loc                                                                     # insertion location

order_min = min(i, j)
order_max = max(i, j)

unchanged_1_indices = np.arange(order_min)                                      # order prior to the smaller index is unchanged
unchanged_2_indices = np.arange(start=(order_max+1), stop=sol_array.shape[0])   # order after the larger index is unchanged
middle_indices = np.arange(order_min, (order_max+1))                            

roll_dir = np.sign(j - i)                                                  
changed = np.roll(middle_indices, roll_dir)                                     # insert the the current node to the insertion location

sol_new_indices = np.concatenate([unchanged_1_indices, changed, unchanged_2_indices])  
sorted_indices = np.argsort(sol_new_indices)                                   # sorted indices by new order
sol_array_new = sol_array[sorted_indices]                                      # new solution

In [22]:
# New score
score + diff

29023994

In [24]:
# Evaluate the result to confirm the insertion
order_series = pd.Series(data=np.arange(len(sol_array_new)), index=sol_array_new)
score_new = utils.evaluate(df, order_series)
score_new

29023994